In [4]:
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool
import os
import shutil
import time
import requests
import re
import json
import glob
import pandas as pd

In [5]:
BASE_URL = 'http://www.winemag.com/?s=&drink_type=wine&page={0}'
session = requests.Session()
HEADERS = {
    'user-agent': ('Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/48.0.2564.109 Safari/537.36')
}
DATA_DIR = 'data'
FILENAME = 'winemag-data'

In [107]:
def scrape_page(wine_name):
    formatted_wine_name = wine_name.replace('\'','')
    formatted_wine_name = formatted_wine_name.replace('(','')
    formatted_wine_name = formatted_wine_name.replace(')','')
    formatted_wine_name = formatted_wine_name.replace(' ','-').lower()
    print(formatted_wine_name)
    
    page_url = 'https://www.winemag.com/buying-guide/{}/'.format(formatted_wine_name)
    
    retry_count = 0

    try:
        response = session.get(page_url, headers=HEADERS)
    except:
        retry_count += 1
        if retry_count <= 3:
            response = session.get(page_url, headers=HEADERS)
        else:
            raise

    soup = BeautifulSoup(response.content, 'html.parser')
    txt = soup.get_text().replace('\n','')
    
    return txt

In [146]:
df = pd.read_csv('sample_2k_wines.csv')
df.head()

,country,description,designation,points,price,province,region_1,title,variety,winery,...,avg_temp,range_temp,stdv_temp,avg_precip,range_precip,stdv_precip,lat,lng,elevation,review_length
0,United States of America,"Tastes unnatural and manipulated, with sugary ...",NaN,82,26.0,California,Napa Valley,Frog's Leap 2010 Chardonnay (Napa Valley),Chardonnay,Frog's Leap,...,6.344341,23.887074,8.534117,70.166944,21.132379,6.204226,39.023468,-84.450465,243,18
1,United States of America,"Fairly simple in appeal, with candied, sweet a...",Poizin,87,25.0,California,Sonoma County,Armida 2009 Poizin Zinfandel (Sonoma County),Zinfandel,Armida,...,6.344341,23.887074,8.534117,70.166944,21.132379,6.204226,38.511080,-122.847339,27,42
2,United States of America,"An extremely fragrant, concentrated and rich C...",NaN,90,70.0,California,Red Hills Lake County,Fortress 2007 Cabernet Sauvignon (Red Hills La...,Cabernet Sauvignon,Fortress,...,6.344341,23.887074,8.534117,70.166944,21.132379,6.204226,39.783730,-100.445882,833,57
3,United States of America,"A bold take on a tough grape, this bottling, w...",Claudia Cuvee,88,35.0,California,Adelaida District,Alta Colina 2013 Claudia Cuvee Marsanne (Adela...,Marsanne,Alta Colina,...,6.344341,23.887074,8.534117,70.166944,21.132379,6.204226,39.783730,-100.445882,833,53
4,Portugal,Finely structured and with a good balance betw...,Quinto Elemento Reserva,89,30.0,Tejo,NaN,Quinta do Arrobe 2012 Quinto Elemento Reserva ...,Syrah,Quinta do Arrobe,...,16.075776,9.483322,3.353981,55.124185,88.228253,29.332227,39.783730,-100.445882,833,33


In [147]:
#df = df.merge(df.title.apply(lambda s: pd.Series({'abv': scrape_page(x), 'feature2':s-1})), left_index=True, right_index=True)

In [148]:
def get_abv(txt):
    abv_txt = re.findall(r'(?<=Alcohol)(.*)(?=%)', txt)[0]
    abv = float(re.findall(r'[0-9]+.?[0-9]+', abv_txt)[0])
    return abv

def get_bottlesize(txt):    
    bottlesize_txt = re.findall(r'(?<=Bottle Size)(.*)(?=ml)', txt)[0]
    bottlesize = float(re.findall(r'[0-9]+',bottlesize_txt)[0])
    return bottlesize

def get_category(txt):
    importer = re.findall(r'(Importer\S)',txt)
    if importer:
        search_string = r'(?<=mlCategory)(.*)(?={})'.format(importer[0])
        print(search_string)
        category = re.findall(search_string, txt)[0]
    else:
        category = re.findall(r'(?<=mlCategory)(.*)(?=Date Published)', txt)[0]
    
    return category

In [149]:
df = df.iloc[:10,:]
df['page_txt'] = df.title.apply(lambda x: scrape_page(x))
df['abv'] = df.page_txt.apply(lambda x: get_abv(x))
df['bottle_size'] = df.page_txt.apply(lambda x: get_bottlesize(x))
df['category'] = df.page_txt.apply(lambda x: get_category(x))

df.head(10)

frogs-leap-2010-chardonnay-napa-valley
armida-2009-poizin-zinfandel-sonoma-county
fortress-2007-cabernet-sauvignon-red-hills-lake-county
alta-colina-2013-claudia-cuvee-marsanne-adelaida-district
quinta-do-arrobe-2012-quinto-elemento-reserva-syrah-tejo
achaia-clauss-2015-demestica-white-moschofilero-peloponnese
quilceda-creek-2007-red-wine-red-columbia-valley-wa
allan-scott-2009-sauvignon-blanc-marlborough
château-les-palais-2004-cuvée-tradition-red-corbières
opolo-2012-serenade-cabernet-sauvignon-malbec-paso-robles


IndexError: list index out of range

In [150]:
df.page_txt.iloc[9]

'Opolo 2012 Serenade Cabernet Sauvignon-Malbec (Paso Robles) Rating and Review | Wine Enthusiast Magazine        var _site_base_url = "https://www.winemag.com";        var _is_search  = "";        var _is_bestBuy = ""    a.article-listing-block.image-size-1536x1536 .bg-image-thumb:after { padding-top: 100%; }a.article-listing-block.image-size-2048x2048 .bg-image-thumb:after { padding-top: 100%; }a.article-listing-block.image-size-related-stories-small .bg-image-thumb:after { padding-top: 65.71%; }a.article-listing-block.image-size-top-picks .bg-image-thumb:after { padding-top: 65.71%; }a.article-listing-block.image-size-mobile .bg-image-thumb:after { padding-top: 65.71%; }a.article-listing-block.image-size-related-stories .bg-image-thumb:after { padding-top: 65.91%; }a.article-listing-block.image-size-top-nav-channel .bg-image-thumb:after { padding-top: 65.91%; }a.article-listing-block.image-size-landing-page .bg-image-thumb:after { padding-top: 65.89%; }a.article-listing-block.image-s

In [93]:
purl = df.title.iloc[4]

In [100]:
wine_name = purl

formatted_wine_name = wine_name.replace('\'','')
formatted_wine_name = formatted_wine_name.replace('(','')
formatted_wine_name = formatted_wine_name.replace(')','')
formatted_wine_name = formatted_wine_name.replace(' ','-').lower()

page_url = 'https://www.winemag.com/buying-guide/{}/'.format(formatted_wine_name)
print(page_url)

retry_count = 0

try:
    response = session.get(page_url, headers=HEADERS)
except:
    retry_count += 1
    if retry_count <= 3:
        response = session.get(page_url, headers=HEADERS)
    else:
        raise

soup = BeautifulSoup(response.content, 'html.parser')
txt = soup.get_text().replace('\n','')

abv_txt = re.findall(r'(?<=Alcohol)(.*)(?=%)', txt)[0]
abv = float(re.findall(r'[0-9]+.?[0-9]+', abv_txt)[0])
category = re.findall(r'(?<=mlCategory)(.*)(?=Date Published)', txt)[0]

abv

https://www.winemag.com/buying-guide/quinta-do-arrobe-2012-quinto-elemento-reserva-syrah-tejo/


14.0

In [66]:
'https://www.winemag.com/buying-guide/frogs-leap-2010-chardonnay-napa-valley/'

'https://www.winemag.com/buying-guide/frogs-leap-2010-chardonnay-napa-valley/'

In [133]:
t = 'Importer Importerz '
if re.findall(r'(Importer\S)', t):
    print('yes')


yes
